In [17]:
import pandas as pd

import json

import datetime

import random

import timeit

import numpy as np

import requests

from citipy import citipy

from config import weather_key

import time

In [18]:
# Perform API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_key

In [19]:
# Create a set of 2000 random values for latitude and longitude
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [20]:
# Add the latitudes and longitudes to a list

coordinates = list(lat_lngs)

In [21]:
# Create a list for holding the cities.

cities = []

# Identify the nearest city for each latitude and longitude combination.

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.

    if city not in cities:
        cities.append(city)
      
len(cities)

797

In [22]:
# Create an empty list to hold the weather data.

city_data = []

# Print the beginning of the logging.

print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.

record_count = 1
set_count = 1

# Loop through all the cities in the list.
                                        #[:10]use after cities to check #entered cities rather than the entire list
for i, city in enumerate(cities[:50]):

    # Group cities in sets of 50 for logging purposes.

    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.

    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.

    print(f"Processing Record {record_count} of Set {set_count} | {city}")
   
    # Add 1 to the record count.

    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.

        city_weather = requests.get(city_url).json()
        # Check this request by using  ## print(json.dumps(city_weather,indent=2))
        
        # Parse out the needed data.

        city_lat = city_weather["coord"]["lat"]
        city_country = city_weather["sys"]["country"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_descrip = city_weather['weather'][0]['description']
        
        
        # Convert the date to ISO standard.

        city_date = datetime.datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
       
        # Append the city information into city_data list.

        city_data.append({"City": city.title(),
                        "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_descrip})
                          

# If an error is experienced, skip the city. Print error 

    except Exception as e:
        print(e)
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.

print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | manaure
Processing Record 2 of Set 1 | punta arenas
Processing Record 3 of Set 1 | namibe
Processing Record 4 of Set 1 | nelson bay
Processing Record 5 of Set 1 | atuona
Processing Record 6 of Set 1 | qui nhon
Processing Record 7 of Set 1 | gizo
Processing Record 8 of Set 1 | severo-kurilsk
Processing Record 9 of Set 1 | dikson
Processing Record 10 of Set 1 | norman wells
Processing Record 11 of Set 1 | coolum beach
Processing Record 12 of Set 1 | talnakh
Processing Record 13 of Set 1 | ushuaia
Processing Record 14 of Set 1 | mar del plata
Processing Record 15 of Set 1 | odda
Processing Record 16 of Set 1 | santa lucia
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | georgetown
Processing Record 19 of Set 1 | mataura
Processing Record 20 of Set 1 | fare
Processing Record 21 of Set 1 | zarubino
Processing Record 22 of Set 1 | nicoya
Processing Record 23 of Set 1 | slave lake

In [23]:
city_data_df = pd.DataFrame(city_data)

city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Manaure,CO,11.7751,-72.4445,79.52,79,100,10.85,overcast clouds
1,Punta Arenas,CL,-53.1500,-70.9167,30.31,93,0,5.75,clear sky
2,Namibe,AO,-15.1961,12.1522,68.63,77,0,8.05,clear sky
3,Nelson Bay,AU,-32.7167,152.1500,60.01,96,60,9.01,broken clouds
4,Atuona,PF,-9.8000,-139.0333,78.24,76,43,19.24,scattered clouds


In [24]:
# Create the output file (CSV).

output_data_file = "../Weather_Database/WeatherPy_Database.csv"

# Export the City_Data into a CSV.

city_data_df.to_csv(output_data_file)